Below are some helper functions to help get you started. They should look familiar from the lesson!

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

In [ ]:
import math
from scipy import stats

# Lane lines of a previous frame in slope, intercept
prev_l_line = (0, 0)
prev_r_line = (0, 0)

def bleach_yellow(img):
    """
    The function overlays white mask over all yellow pixels of the given image
    """
    hsvImg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([80,100,100])
    upper_yellow = np.array([100,255,255])
    ret, grayWhiteMask = cv2.threshold(
                            cv2.cvtColor(
                                cv2.cvtColor(
                                    cv2.bitwise_or(
                                        hsvImg,
                                        hsvImg,
                                        mask=cv2.inRange(hsvImg, lower_yellow, upper_yellow)),
                                    cv2.COLOR_HSV2BGR),
                                cv2.COLOR_BGR2GRAY),
                            1, 255, cv2.THRESH_BINARY)
    return cv2.bitwise_or(img, cv2.cvtColor(grayWhiteMask, cv2.COLOR_GRAY2BGR))

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def screen_lines(lines, min_slope=0.55):
    """
    This is a debug fuction returning all lines matching the given slope criteria.
    NOTE: It's not used in the project.
    """
    print("Screen lines:")
    screened_lines = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            print("    (%d,%d)-(%d,%d), slope %f" % (x1, y1, x2, y2, slope))
            if abs(slope) > min_slope:
                if y2 < y1:
                    screened_lines.append((x1, y1, x2, y2))
                else:
                    screened_lines.append((x2, y2, x1, y1))
    
    return screened_lines

def get_side_lines(lines, min_slope=0.55):
    """
    Takes an array of unordered lines and returns the left and right lines as slope/intercept tuples.

    Input parameter min_slope indicates the minimum absolute slope of lane lines.
    """
    left_x = []
    left_y = []
    right_x = []
    right_y = []
    # Uncomment for debugging.
    #print("Get side lines:")
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                delta_x = 1e-6
            else:
                delta_x = x2 - x1
            slope = (y2 - y1) / delta_x
            # Uncomment for debugging.
            #print("    (%d,%d)-(%d,%d), slope %f" % (x1, y1, x2, y2, slope))
            if abs(slope) > min_slope:
                if y2 < y1:
                    left_x.extend((x1, x2))
                    left_y.extend((y1, y2))
                else:
                    right_x.extend((x2, x1))
                    right_y.extend((y2, y1))
    if left_x:
        l_slope, l_intercept, r_value, p_value, std_err = stats.linregress(left_x, left_y)
    else:
        l_slope, l_intercept = (0, 0)
        
    if right_x:
        r_slope, r_intercept, r_value, p_value, std_err = stats.linregress(right_x, right_y)
    else:
        r_slope, r_intercept = (0, 0)
    
    return (l_slope, l_intercept), (r_slope, r_intercept)

def average_lines(l_line, r_line, a=0.1):
    """
    The function returns averaged lines as a weighted sum of current line koefficients and the line koefficients of
    the previous call.
    """
    global prev_l_line
    global prev_r_line
    
    if prev_l_line == (0, 0) or prev_r_line == (0, 0):
        prev_l_line = l_line
        prev_r_line = r_line
    else:
        if l_line == (0, 0):
            l_line = prev_l_line
        else:
            l_line = a * l_line[0] + (1 - a) * prev_l_line[0], \
                     a * l_line[1] + (1 - a) * prev_l_line[1]
            prev_l_line = l_line

        if r_line == (0, 0):
            r_line = prev_r_line
        else:
            r_line = a * r_line[0] + (1 - a) * prev_r_line[0], \
                     a * r_line[1] + (1 - a) * prev_r_line[1]
            prev_r_line = r_line

    return l_line, r_line

def draw_lines(img, lines, near_sight, far_sight, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    
    Input parameters near_sight and far_sight indicate the near and far distance constraints.
    """
    # Uncomment for debugging.
    #screened_lines = screen_lines(lines)
    #for line in screened_lines:
    #    cv2.line(img, (line[0], line[1]), (line[2], line[3]), [0, 255, 0], 1)

    l_line, r_line = get_side_lines(lines)
    l_line, r_line = average_lines(l_line, r_line)

    # The variable indicates the desired reduction in px of the lane lines to prevent them from crossing.
    # That is usefull to have the mask bigger than practical view, but it may cause crossing of the lane lines.
    length_reduction = 32

    left_y1 = near_sight
    if l_line[0] != 0:
        l_line_slope = l_line[0]
    else:
        l_line_slope = 1e-6

    left_x1 = int((left_y1 - l_line[1]) / l_line_slope)
    left_y2 = far_sight + length_reduction
    left_x2 = int((left_y2 - l_line[1]) / l_line_slope)
    cv2.line(img, (left_x1, left_y1), (left_x2, left_y2), color, thickness)

    right_y1 = near_sight
    if r_line[0] != 0:
        r_line_slope = r_line[0]
    else:
        r_line_slope = 1e-6

    right_x1 = int((right_y1 - r_line[1]) / r_line_slope)
    right_y2 = far_sight + length_reduction
    right_x2 = int((right_y2 - r_line[1]) / r_line_slope)
    cv2.line(img, (right_x1, right_y1), (right_x2, right_y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, near_sight, far_sight):
    """
    `img` should be the output of a Canny transform.
    
    Input parameters near_sight and far_sight indicate the near and far distance constraints.
    
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines, near_sight, far_sight, [255, 0, 0], 8)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def overlay_lines(img, trapezoid_mask):
    """
    Adds translucent red lines indicating the lane lines to the given image.
    
    Input parameter  trapezoid_mask indicates the trapezoid mask cordinates.
    """
    vertices = np.array([[trapezoid_mask[0],
                          trapezoid_mask[1],
                          trapezoid_mask[2],
                          trapezoid_mask[3]]], dtype=np.int32)
    lines = hough_lines(
        region_of_interest(
            canny(
                gaussian_blur(
                    grayscale(bleach_yellow(img)),
                    5),
                50,
                150),
            vertices),
        1, np.pi/180, 35, 40, 20, trapezoid_mask[0][1], trapezoid_mask[1][1])
    
    return weighted_img(lines, img)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os

imgDir = "test_images/"
imgFiles = os.listdir(imgDir)

for imgFile in imgFiles:
    if '.jpg' in imgFile and 'lines_' not in imgFile:
        prev_l_line = (0, 0)
        prev_r_line = (0, 0)
        image = mpimg.imread(imgDir + imgFile)
        imshape = image.shape
        print(imgFile, ':', type(image), 'with dimesions:', imshape)
        trapezoid_mask = ((0, imshape[0]),
                          (int(imshape[1] / 2 - 20), int(imshape[0] / 2 + 20)),
                          (int(imshape[1] / 2 + 20), int(imshape[0] / 2 + 20)),
                          (imshape[1], imshape[0]))
        mpimg.imsave((imgDir + 'lines_' + imgFile), overlay_lines(image, trapezoid_mask))

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from functools import partial

In [ ]:
def process_image(trapezoid_mask, image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

#    The commented out code is for debug puroses.
#    global img_nbr
#    img_nbr += 1
#    img_name = "challenge/image%04d.jpg" % img_nbr
#    print(img_name)
#    result = overlay_lines(image, trapezoid_mask)
#    mpimg.imsave(img_name, result)
#    return result

    return overlay_lines(image, trapezoid_mask)

Let's try the one with the solid white lane on the right first ...

In [ ]:
prev_l_line = (0, 0)
prev_r_line = (0, 0)
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
trapezoid_mask = ((0, clip1.size[1]),
                  (int(clip1.size[0] / 2 - 20), int(clip1.size[1] / 2 + 20)),
                  (int(clip1.size[0] / 2 + 20), int(clip1.size[1] / 2 + 20)),
                  (clip1.size[0], clip1.size[1]))
custom_process_image = partial(process_image, trapezoid_mask)
white_clip = clip1.fl_image(custom_process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
prev_l_line = (0, 0)
prev_r_line = (0, 0)
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
trapezoid_mask = ((0, clip2.size[1]),
                  (int(clip2.size[0] / 2 - 20), int(clip2.size[1] / 2 + 20)),
                  (int(clip2.size[0] / 2 + 20), int(clip2.size[1] / 2 + 20)),
                  (clip2.size[0], clip2.size[1]))
custom_process_image = partial(process_image, trapezoid_mask)
yellow_clip = clip2.fl_image(custom_process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


[YM] Challenge accepted! The current pipeline is using linear reggession to interpolate the points of detected line segments to two lane lines - left and right. I think that a better solution should interpolate the detected points with a cubic spline. Besides the road curves, the most challenging part of the problem was the concrete bridge in challenge.mp4. The pipeline is bleaching the yellow median line on the bridge in order to improve the line gradient. However the pipeline doesn't handle well the black rubber marks on the light bridge surface, while the edges of the marks have a higher gradient than the yellow median line, so they are falsely detected as lane lines. I'm not yet sure how to solve the problem, perhaps by discarding line segments located far from the regions where most of the line segments are detected or where the lines have been detected in previous frames.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
# The commented out global variable is for debug purposes. It indicates the current frame number for saving
# to disk as a picture.
#img_nbr = 0
prev_l_line = (0, 0)
prev_r_line = (0, 0)
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
trapezoid_mask = ((0, clip2.size[1] - 45),
                  (int(clip2.size[0] / 2 - 30), int(clip2.size[1] / 2 + 50)),
                  (int(clip2.size[0] / 2 + 30), int(clip2.size[1] / 2 + 50)),
                  (clip2.size[0], clip2.size[1] - 45))
custom_process_image = partial(process_image, trapezoid_mask)
challenge_clip = clip2.fl_image(custom_process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))